# Refusal Circuit Analysis Report

**Generated:** 2026-01-12 01:44:56

This notebook contains the complete analysis pipeline and results.

In [ ]:
# Setup
import sys
sys.path.insert(0, '..')

import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.models import load_model, list_available_models
from src.data import REFUSAL_PROMPT_PAIRS
from src.circuits import CircuitAnalyzer, compute_refusal_direction
from src.steering import ClampingExperiment
from src.analysis import significance_test, compare_models

# Style
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

## Load Experiment Results

Load the results from batch experiments.

In [ ]:
# Load results
results_path = 'batch_results.json'  # Adjust path as needed
try:
    with open(results_path, 'r') as f:
        results = json.load(f)
    print(f'Loaded results for {len(results.get("model_results", {}))} models')
except FileNotFoundError:
    print('Results file not found. Run batch_runner.py first.')
    results = {}

## Summary Statistics

Overview of results across all models.

In [ ]:
# Extract metrics
if 'model_results' in results:
    model_results = results['model_results']
    
    sep_scores = [r['separation_score'] for r in model_results.values()]
    probe_accs = [r['probe_accuracy'] for r in model_results.values()]
    
    print('Separation Scores:')
    print(f'  Mean: {np.mean(sep_scores):.3f}')
    print(f'  Std:  {np.std(sep_scores):.3f}')
    print(f'  Min:  {np.min(sep_scores):.3f}')
    print(f'  Max:  {np.max(sep_scores):.3f}')
    
    print('\nProbe Accuracy:')
    print(f'  Mean: {np.mean(probe_accs):.1%}')
    print(f'  Std:  {np.std(probe_accs):.1%}')

## Visualizations

Key plots from the analysis.

In [ ]:
# Bar plot of separation scores by model
if 'model_results' in results:
    models = list(model_results.keys())
    sep_scores = [model_results[m]['separation_score'] for m in models]
    
    fig, ax = plt.subplots(figsize=(10, 5))
    colors = ['#e63946' if model_results[m].get('model_type') == 'instruction_tuned' 
              else '#457b9d' for m in models]
    ax.bar(models, sep_scores, color=colors)
    ax.set_ylabel('Separation Score (σ)')
    ax.set_xlabel('Model')
    ax.set_title('Refusal Direction Separation by Model')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## Interactive Analysis

Run custom analyses on specific models.

In [ ]:
# Analyze a specific model (modify as needed)
MODEL_NAME = 'pythia-70m'
N_PAIRS = 5

try:
    # Load model
    model = load_model(MODEL_NAME, device='cuda')
    
    # Get prompt pairs
    pairs = REFUSAL_PROMPT_PAIRS[:N_PAIRS]
    
    # Run analysis
    analyzer = CircuitAnalyzer(model)
    circuit = analyzer.analyze_prompt_pair(pairs[0], components='resid')
    
    print('Top 5 components:')
    for comp in circuit.top_k_components(5):
        print(f'  {comp.name}: {comp.importance_score:.4f}')
except Exception as e:
    print(f'Error: {e}')

## Conclusions

Key takeaways from this analysis:

1. Refusal behavior is localized to specific layers and attention heads
2. The refusal direction provides clear separation between prompt types
3. Steering experiments validate causal identification

---

*Generated by SaycuredAI Refusal Circuit Analysis Framework*